# Lab 03_1: Qubit-Resonator System (3D Cavity)

### modified by Seong Hyeon Park (pajoheji0909@snu.ac.kr)
### original code from pyEPR Tutorial_notebooks (ZK Minev: https://github.com/zlatko-minev/pyEPR)

### Please find the attached homework at the end of this tutorial code

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2

import pyEPR as epr

## connect to the example HFSS project file

In [ ]:
# NOTE: you have to correctly set the project path
project_path =  './' 
# project "file name"
project_name = 'pyEPR_tutorial1'
# project "design name"
design_name  = '1. single_transmon'

pinfo = epr.ProjectInfo(project_path = project_path,
                        project_name = project_name,
                        design_name = design_name)

In [ ]:
# Qubit junction

Lj_value = '12 nH'
Cj_value = '2 fF'

pinfo.design.set_variable('Lj_1', Lj_value); pinfo.design.set_variable('Cj_1', Cj_value)

# NOTE: correctly set the L_j (and C_j), JJ rectangular and line objects
pinfo.junctions['junction'] = {'Lj_variable' : 'Lj_1',
                               'Cj_variable' : 'Cj_1',
                                'rect' : 'rect_jj1',
                                'line' : 'line_jj1'}
pinfo.validate_junction_info()                                

## update the eigenmode simulation setup

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

setup_name = "Setup1"

# delete already existing setup and create a new one
try:
    pinfo.design.delete_setup(setup_name)
except:
    pass

pinfo.design.create_em_setup(name=setup_name,
                             min_freq_ghz=2,
                             n_modes=2,
                             max_delta_f=0.5,
                             max_passes=15,
                             min_passes=1,
                             min_converged=2,
                             basis_order=1)

# check the setup
pinfo.setup

In [ ]:
pinfo.design.save_screenshot()

## Do EPR analysis!

In [ ]:
pinfo.setup.analyze()

epr_hfss = epr.DistributedAnalysis(pinfo)
    
epr_hfss.do_EPR_analysis()

## Do EPR analysis! (numerical diagonalization of the Hamiltonian)

#### NOTE: `cos_trunc`: truncated order for a cosine potential of a Josephson junction and `Fock_trunc`: truncated order for an oscillator   
#### NOTE: `PT`: Perturbation theory and `ND`: Numerical diagonalization

In [ ]:
epra = epr.QuantumAnalysis(epr_hfss.data_filename)
epra.analyze_all_variations(cos_trunc = 10, fock_trunc = 15)

## parametric sweep simulation

Linear step or linear count are useful to define parametric sweep simulations in ANSYS HFSS

In [ ]:
opti_name = "linear_step"
swp_variable = 'height'
swp_params = ('30mm','36mm','1mm')

# 'height' is a geometric variable so cannot copy mesh between passes

sweep_settings = dict(
    variable = swp_variable,
    swp_type = 'linear_step',
    swp_params = swp_params,
    name = opti_name,
    setup_name = setup_name, 
    save_fields = True,
    copy_mesh = False, 
    solve_with_copied_mesh_only = False, 
    setup_type = 'parametric'
)

# setup_name=None will use the first setup
if opti_name not in pinfo.design.optimetrics.get_setup_names():
    opti_setup = pinfo.design.optimetrics.create_setup(**sweep_settings)
    
"""
opti_name = "linear_count"
swp_variable = 'pad_gap'
swp_params = ('80um', '120um', 5)

sweep_settings = dict(
    variable = swp_variable,
    swp_type = 'linear_count',
    swp_params = swp_params,
    name = opti_name,
    setup_name = setup_name, 
    save_fields = True,
    copy_mesh = False, 
    solve_with_copied_mesh_only = False, 
    setup_type = 'parametric'
)

if opti_name not in pinfo.design.optimetrics.get_setup_names():
    opti_setup = pinfo.design.optimetrics.create_setup(**sweep_settings)
"""

In [ ]:
analysis_setup = pinfo.design.get_setup(setup_name)
analysis_setup.solve(setup_name)

pinfo.design.optimetrics.solve_setup("param_file")

# LAB HOMEWORK

### Transmon-cavity system (floating type; double pad geometry) simulations. You should achieve the target parameters within 3\%. The overall circuit layout, electric field distribution, and the convergence plot vs. adaptive pass should be attached to the report.

1. Analyze the variations of qubit frequency $\omega_q$, qubit anharmonicity $\alpha$, cavity resonator frequency $\omega_c$, and qubit-resonator cross Kerr shift $\chi_{qr}$ by increasing (or decreasing) `pad_gap`, `pad_w` and `pad_h` of a double pad geometry and `Lj_1` of a Josephson junction.
2. Analyze the variations of qubit frequency $\omega_q$, qubit anharmonicity $\alpha$, cavity resonator frequency $\omega_c$, and qubit-resonator cross Kerr shift $\chi_{qr}$ by increasing (or decreasing) the geometric features of a cavity.